## Imports and Custom Functions

In [1]:
#Imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import pandas as pd
import spacy
import nltk
import math


from nltk.probability import FreqDist
from wordcloud import WordCloud
pd.set_option('display.max_colwidth', None)

In [2]:
def batch_preprocess_texts(
    texts,
    nlp=None,
    remove_stopwords=True,
    remove_punct=True,
    use_lemmas=False,
    disable=["ner"],
    batch_size=50,
    n_process=-1,
):
    """Efficiently preprocess a collection of texts using nlp.pipe()
    Args:
        texts (collection of strings): collection of texts to process (e.g. df['text'])
        nlp (spacy pipe), optional): Spacy nlp pipe. Defaults to None; if None, it creates a default 'en_core_web_sm' pipe.
        remove_stopwords (bool, optional): Controls stopword removal. Defaults to True.
        remove_punct (bool, optional): Controls punctuation removal. Defaults to True.
        use_lemmas (bool, optional): lemmatize tokens. Defaults to False.
        disable (list of strings, optional): named pipeline elements to disable. Defaults to ["ner"]: Used with nlp.pipe(disable=disable)
        batch_size (int, optional): Number of texts to process in a batch. Defaults to 50.
        n_process (int, optional): Number of CPU processors to use. Defaults to -1 (meaning all CPU cores).
    Returns:
        list of tokens
    """
    # from tqdm.notebook import tqdm
    from tqdm import tqdm
    if nlp is None:
        nlp = spacy.load("en_core_web_sm")
    processed_texts = []
    for doc in tqdm(nlp.pipe(texts, disable=disable, batch_size=batch_size, n_process=n_process)):
        tokens = []
        for token in doc:
            # Check if should remove stopwords and if token is stopword
            if (remove_stopwords == True) and (token.is_stop == True):
                # Continue the loop with the next token
                continue
            # Check if should remove stopwords and if token is stopword
            if (remove_punct == True) and (token.is_punct == True):
                continue
            # Check if should remove stopwords and if token is stopword
            if (remove_punct == True) and (token.is_space == True):
                continue
            
            ## Determine final form of output list of tokens/lemmas
            if use_lemmas:
                tokens.append(token.lemma_.lower())
            else:
                tokens.append(token.text.lower())
        processed_texts.append(tokens)
    return processed_texts


## Load and Check Data

In [3]:
df = pd.read_csv("Data/Fake_Real_News_Data.csv")
df.head()

Unnamed: 0  \
0           0   
1           1   
2           2   
3           3   
4           4   

                                                                             title  \
0     A whirlwind day in D.C. showcases Trump’s unorthodox views and shifting tone   
1  In Baltimore's call for federal police probe, a new search for answers (+video)   
2             Trump Proudly Declares: Most Of The People I’ve Insulted Deserved It   
3               Inside the Trump-Bush melodrama: Decades of tension and discomfort   
4                                    Shutdown clash to return in force by December   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


## Clean Data

In [5]:
df.duplicated().sum()

0

In [6]:
df.isna().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [7]:
# Drop unnecessary columns
df = df.drop(columns=["Unnamed: 0"])

In [8]:
df.head()

title  \
0     A whirlwind day in D.C. showcases Trump’s unorthodox views and shifting tone   
1  In Baltimore's call for federal police probe, a new search for answers (+video)   
2             Trump Proudly Declares: Most Of The People I’ve Insulted Deserved It   
3               Inside the Trump-Bush melodrama: Decades of tension and discomfort   
4                                    Shutdown clash to return in force by December   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## Tokenize and Lemmatize Data

In [9]:
# Split text by spaces to create tokens
df["tokens"] = df["text"].str.split()
df.head()

title  \
0     A whirlwind day in D.C. showcases Trump’s unorthodox views and shifting tone   
1  In Baltimore's call for federal police probe, a new search for answers (+video)   
2             Trump Proudly Declares: Most Of The People I’ve Insulted Deserved It   
3               Inside the Trump-Bush melodrama: Decades of tension and discomfort   
4                                    Shutdown clash to return in force by December   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [10]:
# Create spacy object
nlp_model = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp_model.pipe_names

['tok2vec', 'tagger', 'attribute_ruler', 'lemmatizer']

In [ ]:
# Lemmatize text
df["lemmas"] = batch_preprocess_texts(df["text"])

5412it [01:31, 93.55it/s] 

In [ ]:
df.head()

In [ ]:
# Join lemmas into single string
df["joined_lemmas"] = df["lemmas"].map(lambda x: " ".join(x))
df.head()

## Check Class Balance and Add Length Columnd text

In [ ]:
# Check class balance
df["label"].value_counts()

In [ ]:
# Create length column using token count
df["length"] = df["tokens"].map(len)
df.head()

In [ ]:
# Create variable (typeSeries) for real articles
real_length = df["length"].where(df["label"] == "REAL")

In [ ]:
# Create variable (typeSeries) for fake articles
fake_length = df["length"].where(df["label"] == "FAKE")

In [ ]:
# Check data for real articles. Average length is 873 words
real_length.describe()

In [ ]:
# Check data for fake articles. Average length is 679 words
fake_length.describe()

## Frequency Distribution

In [ ]:
# Create lists for real and fake lemmas
real_words = df["lemmas"].where(df["label"] == "REAL").explode().to_list()

fake_words = df["lemmas"].where(df["label"] == "FAKE").explode().to_list()

In [ ]:
## Instantiate positive frequency distribution
real_freq_dist = FreqDist(real_words)
fake_freq_dist = FreqDist(fake_words)

## Plot the distribution
real_freq_dist.plot(20, title='Real News Lemma Frequency Distribution')

fake_freq_dist.plot(20, title='Fake News Lemma Frequency Distribution');

In [ ]:
# Create lists for real and fake joined lemmas
real_words = df["joined_lemmas"].where(df["label"] == "REAL").explode().to_list()

fake_words = df["joined_lemmas"].where(df["label"] == "FAKE").explode().to_list()

In [ ]:
# https://stackoverflow.com/questions/10880813/typeerror-sequence-item-0-expected-string-int-found
# Create new variable without NaNs (categorized as floats)
real_words_a = ','.join(str(v) for v in real_words)

fake_words_a = ','.join(str(f) for f in fake_words)


## Word Clouds

In [ ]:
# Create an instance of a WordCloud for the real articles
cloud = WordCloud(
    random_state=123,
    stopwords = [],
    background_color="white",
    width = 500,
    height = 500,
    max_words=200,
    colormap="plasma",
    min_word_length=2,
).generate(real_words_a)

plt.figure(figsize = (10, 10))
plt.imshow(cloud)
plt.axis("off");



In [ ]:
# Create an instance of a WordCloud for the fake articles
cloud = WordCloud(
    random_state=123,
    stopwords = [],
    background_color="black",
    width = 500,
    height = 500,
    max_words=200,
    colormap="plasma",
    min_word_length=2,
).generate(fake_words_a)

plt.figure(figsize = (10, 10))
plt.imshow(cloud)
plt.axis("off");

